In [ ]:
import numpy as np
import torch
import torch.nn as nn
import art.estimators as estimators
import art.attacks.evasion as toolbox
from art.estimators.classification import PyTorchClassifier
# from kd_export import LightNN
from mvtec import test_dataset

Organizing dataset...

+------------+-------------------------+------------------------+
| Tool       |   Training Images Moved |   Testing Images Moved |
+============+=========================+========================+
| bottle     |                     209 |                     20 |
+------------+-------------------------+------------------------+
| cable      |                     224 |                     58 |
+------------+-------------------------+------------------------+
| capsule    |                     219 |                     23 |
+------------+-------------------------+------------------------+
| carpet     |                     280 |                     28 |
+------------+-------------------------+------------------------+
| grid       |                     264 |                     21 |
+------------+-------------------------+------------------------+
| hazelnut   |                     391 |                     40 |
+------------+-------------------------+-------------

In [3]:
class LightNN(nn.Module):
    def __init__(self, num_classes=10):
        super(LightNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 56 * 56, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
print("Check current device: ")
# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): # Should return True 
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # Should show your GPU name
else:
    print("Using CPU")

Check current device: 
Using GPU: NVIDIA GeForce RTX 4060


In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95]
pretrained_model = "student_model.pth"
pretrained_model_KD = "student_model_KD.pth"
use_cuda=True
# Set random seed for reproducibility
torch.manual_seed(42)

In [7]:
# Initialize the network
torch.manual_seed(42)
model = LightNN(num_classes=15).to(device)
torch.manual_seed(42)
model_KD = LightNN(num_classes=15).to(device)


# Load the pretrained model
model.load_state_dict(torch.load(pretrained_model, map_location=device, weights_only=True))
model_KD.load_state_dict(torch.load(pretrained_model_KD, map_location=device, weights_only=True))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()
model_KD.eval()

LightNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=50176, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=15, bias=True)
  )
)

In [ ]:
# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Wrap the model with PyTorchClassifier
# classifier = PyTorchClassifier(
#     model=model,
#     clip_values=(0, 1),  # Input values range, e.g., for normalized images
#     loss=loss_fn,
#     optimizer=optimizer,
#     input_shape=(3, 224, 224),  # Adjust to your input shape
#     nb_classes=15              # Number of output classes
# )

classifier = 

In [9]:
x_test = np.array([data[0].numpy() for data in test_dataset])
y_test = np.array([data[1] for data in test_dataset])

In [24]:
# Create FGSM attack
attack = toolbox.FastGradientMethod(estimator=model_KD, eps=0.1)

# Generate adversarial examples
x_test_adv_fgsm = attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_fgsm)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on FSGM attack: {accuracy * 100:.2f}%")

Universal perturbation:   5%|▌         | 2/40 [06:23<2:01:22, 191.65s/it]


EstimatorError: FastGradientMethod requires an estimator derived from <class 'art.estimators.estimator.BaseEstimator'> and <class 'art.estimators.estimator.LossGradientsMixin'>, the provided classifier is an instance of <class '__main__.LightNN'> and is derived from (<class 'torch.nn.modules.module.Module'>,).

In [13]:
# Create PGD attack
pgd_attack = toolbox.ProjectedGradientDescent(estimator=classifier, eps=0.1, eps_step=0.01, max_iter=40)

# Generate adversarial examples
x_test_adv_pgd = pgd_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_pgd)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on PGD attack: {accuracy * 100:.2f}%")

Accuracy on PGD attack: 76.66%


In [14]:
# Create C&W attack
cw2_attack = toolbox.CarliniL2Method(classifier=classifier, confidence=0.0, max_iter=40, learning_rate=0.01)

# Generate adversarial examples
x_test_adv_cw2 = cw2_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_cw2)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on CW2 attack: {accuracy * 100:.2f}%")

C&W L_2:   0%|          | 1/467 [00:16<2:10:48, 16.84s/it]


KeyboardInterrupt: 

In [23]:
# Create Universal Perturbations attack
up_attack = toolbox.UniversalPerturbation(classifier=classifier, attacker="bim",
                                        #   delta=0.2,
                                          max_iter=40,
                                          eps=0.1,
                                        #   batch_size=32,
                                        #   verbose=True
                                        )

# Generate adversarial examples
x_test_adv_up = up_attack.generate(x=x_test)

# Evaluate accuracy on adversarial examples
predictions = classifier.predict(x_test_adv_up)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print(f"Accuracy on UP attack: {accuracy * 100:.2f}%")

KeyboardInterrupt: 